In [57]:
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import numpy as np
import requests
import datetime
import time

In [5]:
url='https://transfermarkt.com/premier-league/transfers/wettbewerb/GB1/plus/?saison_id=2014&s_w=&leihe=0&intern=0'
'https://www.transfermarkt.com/laliga/transfers/wettbewerb/ES1/plus/?saison_id=2014&s_w=&leihe=0&intern=0'
'https://www.transfermarkt.com/{}/transfers/wettbewerb/{}/plus/?saison_id={}&s_w=&leihe=0&intern=0'

In [55]:
leagues=[('premier-league', 'GB1'), ('laliga', 'ES1'), ('bundesliga', 'L1'), ('serie-a', 'IT1'), ('ligue-1', 'FR1')]
start=2014
years=[i for i in range(start, datetime.datetime.now().year)]

[2014, 2015, 2016, 2017, 2018, 2019, 2020]

In [18]:
page=requests.get('https://www.transfermarkt.com/premier-league/transfers/wettbewerb/GB1/plus/?saison_id=2014&s_w=&leihe=0&intern=0', headers={'User-Agent': 'Mozilla/5.0'})
soup=BeautifulSoup(page.content, 'html.parser')

In [34]:
all_teams=soup.find(class_='large-8')

In [35]:
all_teams = all_teams.find_all(class_='box')[3:]

In [92]:
l=[]
for year in years:
    for league in leagues:
        url='https://www.transfermarkt.com/{}/transfers/wettbewerb/{}/plus/?saison_id={}&s_w=&leihe=0&intern=0'.format(league[0],league[1],year)
        print(url)
        page=requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        soup=BeautifulSoup(page.content, 'html.parser')
        all_teams=soup.find(class_='large-8')
        all_teams = all_teams.find_all(class_='box')[3:]
        for team in all_teams:
            team_name=team.find(class_='table-header').h2.a.text
            for index, in_out in enumerate(team.find_all('table')):
                for row in in_out.find_all('tr')[1:]:
                    tds=row.find_all('td')
                    try:
                        player_name=tds[0].div.span.a.text
                        print(player_name, end=' ')
                        age=tds[1].text
                        print(age, end=' ')
                        pos=tds[3].text
                        print(pos, end=' ')
                        if 'Retired' in tds[7].text:
                            from_club = 'retired'
                        else:
                            from_club=tds[7].a.text
                        print(from_club, end=' ')
                        fee=tds[8].a.text
                        print(fee, end=' ')
                        to_club=team_name
                        print(to_club, end=' ')
                        transfer_direction='in' if index == 0 else 'out'
                        season='{}/{}'.format(year,str(year+1)[-2:])
                        print(season, end='')
                        key=player_name+fee+season
                        print(transfer_direction, end=' ')
                        l.append([player_name,age,pos,from_club,to_club,fee,transfer_direction,season,league[0],key])
                        print('')
                    except:
                        print(league[0], year, tds[0].text)

                    
        print(league, year)
        time.sleep(5)

https://www.transfermarkt.com/premier-league/transfers/wettbewerb/GB1/plus/?saison_id=2014&s_w=&leihe=0&intern=0
Alexis Sánchez 25 Centre-Forward Barcelona €42.50m Arsenal FC 2014/15in 
Calum Chambers 19 Right-Back Southampton €20.23m Arsenal FC 2014/15in 
Danny Welbeck 23 Centre-Forward Man Utd €20.00m Arsenal FC 2014/15in 
Gabriel Paulista 24 Centre-Back Villarreal €15.00m Arsenal FC 2014/15in 
Mathieu Debuchy 28 Right-Back Newcastle €15.00m Arsenal FC 2014/15in 
David Ospina 25 Goalkeeper OGC Nice €4.00m Arsenal FC 2014/15in 
Krystian Bielik 17 Defensive Midfield Legia Warszawa €2.25m Arsenal FC 2014/15in 
Thomas Vermaelen 28 Centre-Back Barcelona €19.00m Arsenal FC 2014/15out 
Johan Djourou 27 Centre-Back Hamburger SV €2.80m Arsenal FC 2014/15out 
Ignasi Miquel 21 Centre-Back Norwich €1.90m Arsenal FC 2014/15out 
Thomas Eisfeld 21 Attacking Midfield Fulham €1.50m Arsenal FC 2014/15out 
Kristoffer Olsson 19 Central Midfield FC Midtjylland €1.00m Arsenal FC 2014/15out 
Lukasz Fabians

In [93]:
df = pd.DataFrame(l)

In [94]:
df

,0,1,2,3,4,5,6,7,8,9
0,Alexis Sánchez,25,Centre-Forward,Barcelona,Arsenal FC,€42.50m,in,2014/15,premier-league,Alexis Sánchez€42.50m2014/15
1,Calum Chambers,19,Right-Back,Southampton,Arsenal FC,€20.23m,in,2014/15,premier-league,Calum Chambers€20.23m2014/15
2,Danny Welbeck,23,Centre-Forward,Man Utd,Arsenal FC,€20.00m,in,2014/15,premier-league,Danny Welbeck€20.00m2014/15
3,Gabriel Paulista,24,Centre-Back,Villarreal,Arsenal FC,€15.00m,in,2014/15,premier-league,Gabriel Paulista€15.00m2014/15
4,Mathieu Debuchy,28,Right-Back,Newcastle,Arsenal FC,€15.00m,in,2014/15,premier-league,Mathieu Debuchy€15.00m2014/15
...,...,...,...,...,...,...,...,...,...,...
12519,Didier Desprez,21,Goalkeeper,Paris FC,RC Lens,free transfer,out,2020/21,ligue-1,Didier Desprezfree transfer2020/21
12520,Cyrille Bayala,24,Right Winger,AC Ajaccio,RC Lens,free transfer,out,2020/21,ligue-1,Cyrille Bayalafree transfer2020/21
12521,Manuel Perez,29,Defensive Midfield,Grenoble,RC Lens,?,out,2020/21,ligue-1,Manuel Perez?2020/21
12522,Aleksandar Radovanovic,27,Centre-Back,KV Kortrijk,RC Lens,?,out,2020/21,ligue-1,Aleksandar Radovanovic?2020/21


In [108]:
len(df['Key'].unique())

10044

In [102]:
df.columns=['Player Name', 'Age', 'Position', 'Team 2', 'Team 1', 'Fee', 'Direction', 'Season', 'League', 'Key']

In [109]:
df = df.drop_duplicates(subset='Key', keep='first')

In [111]:
df.to_csv('historic_transfers.csv')